<p align="center">
  <img src="https://huggingface.co/speakleash/Bielik-7B-Instruct-v0.1/raw/main/speakleash_cyfronet.png">
</p>

Notatnik został stworzony, aby umożliwić eksperymentowanie z różnymi rodzajami kwantyzacji modeli. W tym przypadku zaimplementowane zostały trzy przykładowe metody: GPTQ, AWQ i GGUF. Każda z tych metod oferuje unikalne podejście do zmniejszania rozmiaru modelu i wymagań obliczeniowych przy jednoczesnym dążeniu do utrzymania, a nawet zwiększenia wydajności modelu. Dzięki temu interaktywnemu notatnikowi możesz zbadać wpływ tych technik kwantyzacji na różne modele, zapewniając praktyczne doświadczenie z najnowocześniejszą technologią optymalizacji modeli. Niezależnie od tego, czy zamierzasz wdrożyć modele na urządzeniach o ograniczonych zasobach, czy po prostu chcesz poprawić wydajność, ten notatnik służy jako praktyczny przewodnik do zrozumienia i zastosowania tych zaawansowanych metod kwantyzacji.



In [ ]:
%%capture

!pip install accelerate -q
!pip install optimum -q
!pip install auto-gptq -q
!pip install ctransformers[cuda]

In [ ]:
import gc
import torch
import warnings

warnings.filterwarnings("ignore") # zastosowane w celach demonstracyjnych

## **GPTQ**
Gradient Projection for Quantization (GPTQ)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id = "speakleash/Bielik-7B-Instruct-v0.1-GPTQ"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=False,
    revision="main"
)

pipe = pipeline(model=model, tokenizer=tokenizer, task='text-generation')
gptq=True

tokenizer_config.json:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
human_prompt = "Wymień trzy ciekawe miejscowości w Polsce i krótko je scharakteryzuj?"
prompt = f"<s>[INST]{human_prompt}[/INST]"
print(prompt)

<s>[INST]Wymień trzy ciekawe miejscowości w Polsce i krótko je scharakteryzuj?[/INST]


In [ ]:
outputs = pipe(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.1,
    top_p=0.95
)
print(outputs[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s>[INST]Wymień trzy ciekawe miejscowości w Polsce i krótko je scharakteryzuj?[/INST]  1. Kazimierz Dolny: Miasteczko położone nad Wisłą, słynące z malowniczych ruin zamku, wąwozów lessowych i uroczego rynku. Znane z letnich festiwali i zimowych imprez.

2. Gdańsk: Historyczne miasto z portem, położone nad Morzem Bałtyckim. Znane z zabytkowej architektury, Starego Miasta i Bazyliki Mariackiej.

3. Wrocław: Stolica Dolnego Śląska, znana z zabytkowej architektury, Mostu Grunwaldzkiego i wielu parków. Wrocław jest również znany z festiwali kulturalnych i Mostu Zakochanych.


## **GGUF**
Group-wise Gradient Update Filtering (GGUF)

In [ ]:
# poniższy kod wykona sie jedynie w przypadku, gdy tokenizer został uprzednio 'zainicjalizowany'
if 'model' in globals() and 'tokenizer' in globals() and 'pipe' in globals():
  print("Remove from memory: model, tokenizer and pipe")
  del model, tokenizer, pipe
  gc.collect()
  torch.cuda.empty_cache()

In [ ]:
from ctransformers import AutoModelForCausalLM
from transformers import AutoTokenizer, pipeline

model = AutoModelForCausalLM.from_pretrained(
    "speakleash/Bielik-7B-Instruct-v0.1-GGUF",
    model_file="bielik-7b-instruct-v0.1.Q4_K_M.gguf", # różne typy kwantyzacji można pobrać z repozytorium speakleash/Bielik-7B-Instruct-v0.1-GGUF
    model_type="mistral", gpu_layers=50, hf=True
)
tokenizer = AutoTokenizer.from_pretrained(
    "speakleash/Bielik-7B-Instruct-v0.1", use_fast=True
)

pipe = pipeline(model=model, tokenizer=tokenizer, task='text-generation')

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

bielik-7b-instruct-v0.1.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
human_prompt = "Wymień trzy ciekawe miejscowości w Polsce i krótko je scharakteryzuj?"
prompt = f"<s>[INST]{human_prompt}[/INST]"
print(prompt)

<s>[INST]Wymień trzy ciekawe miejscowości w Polsce i krótko je scharakteryzuj?[/INST]


In [ ]:
outputs = pipe(prompt, max_new_tokens=256)
print(outputs[0]["generated_text"])

<s>[INST]Wymień trzy ciekawe miejscowości w Polsce i krótko je scharakteryzuj?[/INST]  Trzy ciekawe miejscowości w Polsce to:

1. Kraków - dawna stolica Polski, znana z zabytkowej architektury, w tym Wawelu, Sukiennic i Kościoła Mariackiego. Kraków jest również centrum kultury, oferującym festiwale, muzea i teatry.

2. Gdańsk - nadmorskie miasto z długą historią. Znane z Bazyliki Mariackiej, Dworu Artusa i fontanny Neptuna. Gdańsk jest również ważnym portem morskim i centrum biznesu.

3. Wrocław - miasto położone na Odrze, znane z licznych mostów i zabytkowej architektury. Wrocław jest Europejską Stolicą Kultury i słynie z festiwali, takich jak Wrocławski Festiwal Filmowy. Miasto jest również domem dla bogatej sieci park


## **AWQ**
Adaptive Weight Quantization (AWQ)

**UWAGA**: Aby użyć poniższego przykładu `AWQ`, należy uprzednio odłączyć środowisko wykonawcze przed zainstalowaniem zależności `vllm`. Mogą wystąpić konflikty zależności z pakietami, które zainstalowaliśmy wcześniej, więc najlepiej zacząć od zera.

In [ ]:
# restart środowiska Colab
exit()

In [ ]:
%%capture

!pip install vllm

In [ ]:
from vllm import LLM, SamplingParams

sampling_params = SamplingParams(temperature=0.0, top_p=1.0, max_tokens=256)
llm = LLM(
    model="speakleash/Bielik-7B-Instruct-v0.1-AWQ",
    quantization='awq',
    dtype='half',
    gpu_memory_utilization=.95,
    max_model_len=4096
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/817 [00:00<?, ?B/s]

WARNING 04-04 10:18:40 config.py:211] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 04-04 10:18:40 llm_engine.py:74] Initializing an LLM engine (v0.4.0.post1) with config: model='speakleash/Bielik-7B-Instruct-v0.1-AWQ', tokenizer='speakleash/Bielik-7B-Instruct-v0.1-AWQ', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=True, quantization=awq, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)


tokenizer_config.json:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

INFO 04-04 10:18:47 selector.py:40] Cannot use FlashAttention backend for Volta and Turing GPUs.
INFO 04-04 10:18:47 selector.py:25] Using XFormers backend.
INFO 04-04 10:18:51 weight_utils.py:177] Using model weights format ['*.safetensors']


model.safetensors:   0%|          | 0.00/4.15G [00:00<?, ?B/s]

INFO 04-04 10:19:39 model_runner.py:104] Loading model weights took 3.8745 GB
INFO 04-04 10:19:43 gpu_executor.py:94] # GPU blocks: 1035, # CPU blocks: 2048
INFO 04-04 10:19:47 model_runner.py:791] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 04-04 10:19:47 model_runner.py:795] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 04-04 10:20:02 model_runner.py:867] Graph capturing finished in 15 secs.


In [ ]:
human_prompt = "Wymień trzy ciekawe miejscowości w Polsce i krótko je scharakteryzuj?"
prompt = f"<s>[INST]{human_prompt}[/INST]"
print(prompt)

<s>[INST]Wymień trzy ciekawe miejscowości w Polsce i krótko je scharakteryzuj?[/INST]


In [ ]:
output = llm.generate(prompt, sampling_params)
print(output[0].outputs[0].text)

Processed prompts: 100%|██████████| 1/1 [00:05<00:00,  5.84s/it]

  Trzy ciekawe miejscowości w Polsce to:
1. Gdańsk - nadmorskie miasto z bogatą historią i kulturą, znane z Bazyliki Mariackiej, Dworu Artusa, fontanny Neptuna i Muzeum II Wojny Światowej.
2. Kraków - historyczne miasto z piękną starówką, Wawelskim Zamkiem, Sukiennicami i Kościołem Mariackim.
3. Wrocław - miasto z malowniczymi kanałami, Starym Rynkiem i zabytkami takimi jak Ostrów Tumski, Most Pokoju i Halą Stulecia.
